In [165]:
import numpy as np
import csv
import sys
import os

import matplotlib
import matplotlib.pyplot as plt

%matplotlib inline

In [166]:
def plot_background_fct(send_data, recv_data, label, do_plot=True):
    # diffs in miliseconds
    diffs = (recv_data - send_data) * 1000
    if do_plot:
        fig, ax1 = plt.subplots(nrows=1, ncols=1, figsize=(10,5))
        ax1.plot(diffs, label="background FCT"+label) 
        ax1.set_title("Background FCT")
        ax1.legend(loc='upper left')
        ax1.set_ylabel('Miliseconds')
        ax1.set_xlabel('Request ID')
        ax1.grid(which='major', linestyle="-")
        ax1.grid(which='minor', linestyle='--')

        fig.tight_layout()   

        fig.savefig("Queuing delay vs Time.png", format="png")
        fig.show()
    return np.mean(diffs), np.percentile(diffs,99)

In [167]:
def plot_foreground_fct(send_data, recv_data, label, do_plot=True):
    # diffs in micrseconds
    diffs = (recv_data - send_data) * 1000 * 1000
    if do_plot:
        fig, ax1 = plt.subplots(nrows=1, ncols=1, figsize=(10,5))
        ax1.plot(diffs, label="background FCT"+label) 
        ax1.set_title("Foreground FCT")
        ax1.legend(loc='upper left')
        ax1.set_ylabel('Microseconds')
        ax1.set_xlabel('Request ID')
        ax1.grid(which='major', linestyle="-")
        ax1.grid(which='minor', linestyle='--')

        fig.tight_layout()   

        fig.savefig("Queuing delay vs Time.png", format="png")
        fig.show()
    return np.mean(diffs), np.percentile(diffs,99)

In [168]:
def plot_queue_size(data, label, do_plot=True):
    # http://www.mathcs.emory.edu/~cheung/Courses/558-old/Syllabus/90-NS/trace.html
    # Columns are: Time(s)-fromNode-toNode-SizeB-SizePack-Packsarrived(in interval)
    # -departed - dropped -3: same in bytes
    times = data[:,0]
    q_size_B, q_size_P = data[:,3], data[:,4]
    q_size_B = np.trim_zeros(q_size_B, trim='b')
    q_size_P = np.trim_zeros(q_size_P, trim='b')
    times = times[:q_size_B.shape[0]]
    
    q_size_B = np.trim_zeros(q_size_B, trim='f')
    q_size_P = np.trim_zeros(q_size_P, trim='f')
    
    times = times[times.shape[0] - q_size_B.shape[0]:]
    
    if do_plot:
        fig, (ax1,ax2) = plt.subplots(nrows=2, ncols=1, figsize=(10,10))
        ax1.plot(times, q_size_B, label="q_size in bytes "+label) 
        ax1.set_title("Q size vs Time")
        ax1.legend(loc='upper left')
        ax1.set_ylabel('Bytes')
        ax1.set_xlabel('Seconds')
        ax1.grid(which='major', linestyle="-")
        ax1.grid(which='minor', linestyle='--')

        ax2.plot(times, q_size_P, label="q_size in packets "+label) 
        ax2.set_title("Q size vs Time")
        ax2.legend(loc='upper left')
        ax2.set_ylabel('Packets')
        ax2.set_xlabel('Seconds')
        ax2.grid(which='major', linestyle="-")
        ax2.grid(which='minor', linestyle='--')

        fig.tight_layout()   

        fig.savefig("Q_size vs Time.png", format="png")
        fig.show()
    #avg_q_occup = np.mean(q_size_B)
    #print("Mon_mean: " + str(avg_q_occup))
    q_size_B = q_size_B/1000.0
    return np.mean(q_size_B), np.mean(q_size_P), np.percentile(q_size_B,99), np.percentile(q_size_P, 99)

In [169]:
def plot_queuing_time(data, label, link_speed, do_plot=True):
    """
    Link speed in Gbps
    results in microseconds
    """
    times = data[:,0]
    q_size_B, q_size_P = data[:,3], data[:,4]
    q_size_B = np.trim_zeros(q_size_B, trim='b')
    times = times[:q_size_B.shape[0]]
    q_size_B = np.trim_zeros(q_size_B, trim='f')
    times = times[times.shape[0] - q_size_B.shape[0]:]
    
    link_speed = link_speed * 1000.0 * 1000.0 * 1000.0
    q_time = q_size_B/(link_speed/8.0) * 1000.0 * 1000.0
    
    if do_plot:
        fig, ax1 = plt.subplots(nrows=1, ncols=1, figsize=(10,5))
        ax1.plot(times, q_time, label="queuing delay "+label) 
        ax1.set_title("Queuing delay vs Time")
        ax1.legend(loc='upper left')
        ax1.set_ylabel('Queuing microseconds')
        ax1.set_xlabel('Simulation Seconds')
        ax1.grid(which='major', linestyle="-")
        ax1.grid(which='minor', linestyle='--')

        fig.tight_layout()   

        fig.savefig("Queuing delay vs Time.png", format="png")
        fig.show()
    return np.mean(q_time), np.percentile(q_time, 99)

In [170]:
def print_results():
    print("Mean Q length in Kbytes {0} and in packets {1}\n99th percentile in Kbytes {2} and in packets {3}\n".format(bytes_mean, pkts_mean, bytes_99, pkts_99))
    print("Mean delay in us {0} and 99th percentile delay in us {1}\n".format(delay_mean, delay_99))
    print("Background FCT (ms): mean: {0}, 99th %: {1}\n".format(bkg_mean, bkg_99))
    print("Foreground FCT (us): mean: {0}, 99th %: {1}\n".format(frg_mean, frg_99))
    
    #print("Mean thrpt in Mbps {0} and 99th percentile thrpt in Mbps {1}".format(thrpt_mean, thrpt_99))    

# Paper reproduction
## 20% bottlenecked link load 

In [171]:
results_folders = ["/TCP/", "/DCTCP30/", "/DCTCP6_pacer/", "/DCTCP_pacer_PQ/"]
latency = dict()
bkg_fct = dict()
frg_fct = dict()

for folder in results_folders:
    latency[folder] = []
    bkg_fct[folder] = []
    frg_fct[folder] = []

In [172]:
simul_folder = "nofanout_20_9flows"

for folder in results_folders:
    path = simul_folder + folder
    q_mon = np.genfromtxt(path+"q_mon", delimiter=" ")
    bytes_mean, pkts_mean, bytes_99, pkts_99 = plot_queue_size(q_mon, folder, do_plot=False)
    delay_mean, delay_99 = plot_queuing_time(q_mon, folder, 1, do_plot=False)
    latency[folder].append((delay_mean, delay_99))
    
    bkg_send = np.genfromtxt(path+"send_times|background.csv", delimiter=" ")
    bkg_recv = np.genfromtxt(path+"rec_times|background.csv", delimiter=" ")
    bkg_mean, bkg_99 = plot_background_fct(bkg_send, bkg_recv, folder, do_plot=False)
    bkg_fct[folder].append((bkg_mean,bkg_99))
    
    frg_send = np.genfromtxt(path+"send_times|foreground.csv", delimiter=" ")
    frg_recv = np.genfromtxt(path+"rec_times|foreground.csv", delimiter=" ")
    frg_mean, frg_99 = plot_foreground_fct(frg_send, frg_recv, folder, do_plot=False)
    frg_fct[folder].append((frg_mean,frg_99))
    print(folder)
    print_results()

/TCP/
Mean Q length in Kbytes 10.373207161667441 and in packets 10.480113982743369
99th percentile in Kbytes 332.04328780830076 and in packets 335.61671041319187

Mean delay in us 82.98565729333953 and 99th percentile delay in us 2656.346302466406

Background FCT (ms): mean: 99.42041444222852, 99th %: 256.5161160532702

Foreground FCT (us): mean: 134.82512828633102, 99th %: 2507.4869754893543

/DCTCP30/
Mean Q length in Kbytes 0.8046506744440401 and in packets 0.8129959245530393
99th percentile in Kbytes 16.604022091903797 and in packets 16.77584842192916

Mean delay in us 6.437205395552321 and 99th percentile delay in us 132.8321767352304

Background FCT (ms): mean: 99.6936984592224, 99th %: 248.5920098640521

Foreground FCT (us): mean: 64.81182661312067, 99th %: 892.2102202782862

/DCTCP6_pacer/
Mean Q length in Kbytes 0.46437153973331313 and in packets 0.4691948669040267
99th percentile in Kbytes 14.518799832816631 and in packets 14.665454376582435

Mean delay in us 3.71497231786650

## 40% bottlenecked link load 

In [173]:
simul_folder = "nofanout_40_9flows"

for folder in results_folders:
    path = simul_folder + folder
    q_mon = np.genfromtxt(path+"q_mon", delimiter=" ")
    bytes_mean,pkts_mean,bytes_99,pkts_99 = plot_queue_size(q_mon, folder, do_plot=False)
    delay_mean, delay_99 = plot_queuing_time(q_mon, folder, 1, do_plot=False)
    latency[folder].append((delay_mean, delay_99))
    
    
    bkg_send = np.genfromtxt(path+"send_times|background.csv", delimiter=" ")
    bkg_recv = np.genfromtxt(path+"rec_times|background.csv", delimiter=" ")
    bkg_mean, bkg_99 = plot_background_fct(bkg_send, bkg_recv, folder, do_plot=False)
    bkg_fct[folder].append((bkg_mean,bkg_99))
    
    
    frg_send = np.genfromtxt(path+"send_times|foreground.csv", delimiter=" ")
    frg_recv = np.genfromtxt(path+"rec_times|foreground.csv", delimiter=" ")
    frg_mean, frg_99 = plot_foreground_fct(frg_send, frg_recv, folder, do_plot=False)
    frg_fct[folder].append((frg_mean,frg_99))
    
    
    
    print(folder)
    print_results()

/TCP/
Mean Q length in Kbytes 46.33385254516568 and in packets 46.81620333165597
99th percentile in Kbytes 447.91081075531605 and in packets 452.43516237910563

Mean delay in us 370.67082036132547 and 99th percentile delay in us 3583.286486042529

Background FCT (ms): mean: 128.396040813339, 99th %: 379.12162808523345

Foreground FCT (us): mean: 436.6546068447897, 99th %: 3628.119887384609

/DCTCP30/
Mean Q length in Kbytes 3.4054378609665275 and in packets 3.440975525760342
99th percentile in Kbytes 26.977927453860868 and in packets 27.252762276541965

Mean delay in us 27.24350288773222 and 99th percentile delay in us 215.82341963088695

Background FCT (ms): mean: 131.49521005674723, 99th %: 371.0459144139071

Foreground FCT (us): mean: 81.91170788224368, 99th %: 867.2783485570801

/DCTCP6_pacer/
Mean Q length in Kbytes 1.954838631667047 and in packets 1.9752280942065399
99th percentile in Kbytes 20.131123971426597 and in packets 20.358160126736664

Mean delay in us 15.638709053336376

## 60% bottlenecked link load

In [174]:
simul_folder = "nofanout_60_9flows"

for folder in results_folders:
    path = simul_folder + folder
    q_mon = np.genfromtxt(path+"q_mon", delimiter=" ")
    bytes_mean,pkts_mean,bytes_99,pkts_99 = plot_queue_size(q_mon, folder, do_plot=False)
    delay_mean, delay_99 = plot_queuing_time(q_mon, folder, 1, do_plot=False)
    latency[folder].append((delay_mean, delay_99))
    
    
    bkg_send = np.genfromtxt(path+"send_times|background.csv", delimiter=" ")
    bkg_recv = np.genfromtxt(path+"rec_times|background.csv", delimiter=" ")
    bkg_mean, bkg_99 = plot_background_fct(bkg_send, bkg_recv, folder, do_plot=False)
    bkg_fct[folder].append((bkg_mean,bkg_99))
    
    
    frg_send = np.genfromtxt(path+"send_times|foreground.csv", delimiter=" ")
    frg_recv = np.genfromtxt(path+"rec_times|foreground.csv", delimiter=" ")
    frg_mean, frg_99 = plot_foreground_fct(frg_send, frg_recv, folder, do_plot=False)
    frg_fct[folder].append((frg_mean,frg_99))
    
    
    print(folder)
    print_results()

/TCP/
Mean Q length in Kbytes 102.93869991844 and in packets 104.02326231856277
99th percentile in Kbytes 467.68113444670985 and in packets 472.72295018572885

Mean delay in us 823.50959934752 and 99th percentile delay in us 3741.449075573679

Background FCT (ms): mean: 197.62561315903577, 99th %: 722.1564153377551

Foreground FCT (us): mean: 910.424324685779, 99th %: 3828.87042060176

/DCTCP30/
Mean Q length in Kbytes 7.1697754579853905 and in packets 7.245438124910812
99th percentile in Kbytes 29.875326871716396 and in packets 30.177476852739492

Mean delay in us 57.35820366388313 and 99th percentile delay in us 239.0026149737312

Background FCT (ms): mean: 210.49253502807503, 99th %: 720.2973403536303

Foreground FCT (us): mean: 123.398575053836, 99th %: 923.0610649630933

/DCTCP6_pacer/
Mean Q length in Kbytes 3.674927298568049 and in packets 3.7137883101678475
99th percentile in Kbytes 22.56359540750519 and in packets 22.81206754250741

Mean delay in us 29.39941838854439 and 99th 

In [175]:
print(latency)
print("------------------")
print(bkg_fct)
print("------------------")
print(frg_fct)

{'/TCP/': [(82.98565729333953, 2656.346302466406), (370.67082036132547, 3583.286486042529), (823.50959934752, 3741.449075573679)], '/DCTCP30/': [(6.437205395552321, 132.8321767352304), (27.24350288773222, 215.82341963088695), (57.35820366388313, 239.0026149737312)], '/DCTCP6_pacer/': [(3.714972317866505, 116.15039866253306), (15.638709053336376, 161.04899177141277), (29.39941838854439, 180.50876326004152)], '/DCTCP_pacer_PQ/': [(0.26555107657600197, 6.760404680174323), (2.45257585693708, 42.67585787628951), (11.911435325310704, 106.08009786239639)]}
------------------
{'/TCP/': [(99.42041444222852, 256.5161160532702), (128.396040813339, 379.12162808523345), (197.62561315903577, 722.1564153377551)], '/DCTCP30/': [(99.6936984592224, 248.5920098640521), (131.49521005674723, 371.0459144139071), (210.49253502807503, 720.2973403536303)], '/DCTCP6_pacer/': [(101.37527114929063, 243.1748274497778), (138.21359499655117, 378.96174769714867), (219.82675888860163, 716.0909031640354)], '/DCTCP_pace

In [176]:
# fig, (ax1,ax2,ax3) = plt.subplots(nrows=1, ncols=3, figsize=(16,10))
# for i, setup in enumerate(results_folders):
#     ax1.bar(i-0.4, latency[setup][0][0], width=0.1, log=True, color="blue")
#     ax1.bar(i-0.3, latency[setup][0][1], width=0.1, log=True, color="blue")
#     ax1.bar(i-0.1, latency[setup][1][0], width=0.1, log=True, color="red")
#     ax1.bar(i, latency[setup][1][1], width=0.1, log=True, color="red")
#     ax1.bar(i+0.2, latency[setup][2][0], width=0.1, log=True, color="green")
#     ax1.bar(i+0.3, latency[setup][2][1], width=0.1, log=True, color="green")
# ax1.set_title("Switch Latency")
# ax1.legend(loc='upper left')
# ax1.set_ylabel('Queuing miliseconds')
# ax1.set_xlabel('Scheme')
# ax1.grid(which='major', linestyle="-")
# #ax1.grid(which='minor', linestyle='--')

# fig.tight_layout()   

# fig.savefig("Queuing delay vs Time.png", format="png")
# fig.show()